[bilibili视频](https://www.bilibili.com/video/av62325981)

[video on youtube](https://www.youtube.com/watch?v=nAl9I88l6so)


In [62]:
from init import *
#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])
config.main_token='UUOS'
wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')
db.reset()
code = r'''
#include <eosio/print.hpp>
#include <eosio/name.hpp>
#include <eosio/action.hpp>
#include <eosio/multi_index.hpp>

#include <string>

using namespace std;
using namespace eosio;

struct record {
   uint64_t  primary;
   string    data;
   uint64_t primary_key() const { return primary; }
   EOSLIB_SERIALIZE( record, (primary)(data))
};

struct getvalue {
   uint64_t  primary;
   uint64_t primary_key() const { return primary; }
   EOSLIB_SERIALIZE( getvalue, (primary))
};


template<typename T>
T unpack_args() {
    size_t raw_args_size = action_data_size();
    check(raw_args_size > 0, "bad args");
    vector<char> raw_args(raw_args_size);
    read_action_data(raw_args.data(), raw_args_size);
    T rec = eosio::unpack<T>(raw_args);
    return rec;
}

extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        if ("setvalue"_n.value == action) {
            uint64_t scope = code;
            uint64_t payer = receiver;
            multi_index<"record"_n, record> mytable(name(code), scope);
            
            record rec = unpack_args<record>();
            auto itr = mytable.find(rec.primary);
            
            if (mytable.end() == itr) {
                mytable.emplace( name(payer), [&]( auto& row ) {
                    row = rec;
                });
            } else {
                mytable.modify( itr, name(payer), [&]( auto& row ) {
                    row.data = rec.data;
                });
            }
        } else if ("getvalue"_n.value == action) {
            uint64_t scope = code;
            uint64_t payer = receiver;
            multi_index<"record"_n, record> mytable(name(code), scope);
            getvalue rec = unpack_args<getvalue>();
            auto itr = mytable.find(rec.primary);
            auto packed_rec = eosio::pack<record>(*itr);
            eosio::printhex(packed_rec.data(), packed_rec.size());
        }
    }
}
'''

abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [],
  "structs": [{
      "name": "record",
      "base": "",
      "fields": [
        {"name":"primary", "type":"uint64"},
        {"name":"data", "type":"string"},
       ]
    },{
      "name": "getvalue",
      "base": "",
      "fields": [
        {"name":"primary", "type":"uint64"},
       ]
    }
  ],
  "actions": [{
      "name": "getvalue",
      "type": "getvalue",
      "ricardian_contract": ""
    },{
      "name": "setvalue",
      "type": "record",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ]
}
'''

publish_cpp_contract('helloworld11', code, abi)
print('done!')




done!


In [63]:
name = 'helloworld11'
try:
    args = {'primary':11, 'data':'hello,worldddd'}
    r = eosapi.push_action(name, 'setvalue', args, {name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)


1358


In [64]:
name = 'helloworld11'
args = eosapi.pack_args(name, 'getvalue', {'primary':11})
eosapi.call_contract(name, 'getvalue', args.hex())

{
    "results": {
        "output": "0b000000000000000e68656c6c6f2c776f726c64646464"
    }
}

In [65]:
name = 'helloworld11'
args = eosapi.pack_args(name, 'getvalue', {'primary':11})
ret = eosapi.call_contract(name, 'getvalue', args.hex())
args = ret['results']['output']
eosapi.unpack_args(name, 'setvalue', bytes.fromhex(args))

{'primary': 11, 'data': 'hello,worldddd'}

In [69]:
from init import *
#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])
config.main_token='UUOS'
wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')
db.reset()
code = r'''
#include <eosio/print.hpp>
#include <eosio/name.hpp>
#include <eosio/action.hpp>
#include <eosio/multi_index.hpp>

#include <string>

using namespace std;
using namespace eosio;

struct record {
   uint64_t  primary;
   string    data;
   uint64_t primary_key() const { return primary; }
   EOSLIB_SERIALIZE( record, (primary)(data))
};

extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        if ("getvalue"_n.value == action) {
            uint64_t scope = code;
            uint64_t payer = receiver;
            multi_index<"record"_n, record> mytable(name(code), scope);
            mytable.emplace( name(payer), [&]( auto& row ) {

            });

        }
    }
}
'''

publish_cpp_contract('helloworld11', code)
print('done!')




done!


In [70]:
eosapi.call_contract(name, 'getvalue', '')

{
    "results": {
        "error": {
            "name": "eosio_assert_message_exception",
            "code": 3050003,
            "what": "eosio_assert_message assertion failure (3050003)\nassertion failure with message: db_store_i64\n"
        }
    }
}

In [78]:
from init import *
#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])
config.main_token='UUOS'
wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')
db.reset()
code = r'''
#include <eosio/print.hpp>
#include <eosio/name.hpp>
#include <eosio/action.hpp>
#include <eosio/system.hpp>
#include <eosio/multi_index.hpp>

#include <string>

using namespace std;
using namespace eosio;

struct record {
   uint64_t  primary;
   string    data;
   uint64_t primary_key() const { return primary; }
   EOSLIB_SERIALIZE( record, (primary)(data))
};

extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        if ("getvalue"_n.value == action) {
            eosio::print(current_time_point().sec_since_epoch());
        }
    }
}
'''

publish_cpp_contract('helloworld11', code)
print('done!')



done!


In [79]:
name = 'helloworld11'
eosapi.call_contract(name, 'getvalue', '')

{
    "results": {
        "error": {
            "name": "eosio_assert_message_exception",
            "code": 3050003,
            "what": "eosio_assert_message assertion failure (3050003)\nassertion failure with message: current_time\n"
        }
    }
}

In [1]:
from init import *
#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])
config.main_token='UUOS'
wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')
db.reset()
code = r'''
#include <eosio/print.hpp>
#include <eosio/name.hpp>
#include <eosio/action.hpp>
#include <eosio/system.hpp>
#include <eosio/multi_index.hpp>

#include <string>

using namespace std;
using namespace eosio;

struct record {
   uint64_t  primary;
   string    data;
   uint64_t primary_key() const { return primary; }
   EOSLIB_SERIALIZE( record, (primary)(data))
};

extern "C" {
    void apply( uint64_t receiver, uint64_t code, uint64_t action ) {
        if ("getvalue"_n.value == action) {
            while(true){};
        }
    }
}
'''

publish_cpp_contract('helloworld11', code)
print('done!')



done!


In [3]:
name = 'helloworld11'
eosapi.call_contract(name, 'getvalue', '')

{
    "results": {
        "error": {
            "name": "deadline_exception",
            "code": 3080006,
            "what": "Transaction took too long (3080006)\ndeadline exceeded!\nrethrow\n"
        }
    }
}